In [ ]:
# import libraries

In [ ]:
# Import dataset, train test split, standardization etc.

In [ ]:
# Place model here

In [ ]:
# Reseting weights after each iteration of looped training/predictions
def reset_weights(model):
    for layer in model.layers:
        if isinstance(layer, tf.keras.Model): #if you're using a model as a layer
            reset_weights(layer) #apply function recursively
            continue

        #where are the initializers?
        if hasattr(layer, 'cell'):
            init_container = layer.cell
        else:
            init_container = layer

        for key, initializer in init_container.__dict__.items():
            if "initializer" not in key: #is this item an initializer?
                  continue #if no, skip it

            # find the corresponding variable, like the kernel or the bias
            if key == 'recurrent_initializer': #special case check
                var = getattr(init_container, 'recurrent_kernel')
            else:
                var = getattr(init_container, key.replace("_initializer", ""))

            var.assign(initializer(var.shape, var.dtype))
            #use the initializer

In [ ]:
# Run the loop
res=[]
std=[]

for i in range(100):

    history = model.fit(x=X_train, y={"retmean_output": ym_train},
                    batch_size=batch_size,
                    epochs=epochs)
    
    pred = model.predict(X_test)
    res.append(pred)
    stdb = np.std(res)
    std.append(stdb)
    
    model.reset_states()
    reset_weights(model)

In [ ]:
# reshape results
resmean = pd.DataFrame(list(map(np.ravel, res)))
resmeanm = resmean.mean(axis=0)
resmeanm = np.array(resmeanm)
resmeanm = np.reshape(resmeanm, (resmeanm.shape[0], 1))

In [ ]:
# Show how the predictions converge
plt.figure(figsize=(7, 2), dpi=80)

plt.plot(std, color = 'black')
plt.title('American Express Co.')
plt.xlabel('Number of iterations')
plt.ylabel('Standard deviation')
plt.show()

In [ ]:
# Look at the results
diff_res = np.subtract(ym_test, resmeanm)
sq_res = np.square(diff_res)
mse_res = sq_res.mean()

print(mse_res)

In [ ]:
# look at the results graphically
plt.plot(ym_test, color = 'black', label = 'Real ret.MidQ mean')
plt.plot(resmeanm, color = 'green', label = 'Predicted ret.MidQ mean')
plt.title('ret.MidQ mean Prediction')
plt.xlabel('Days')
plt.ylabel('scaled ret.MidQ mean')
plt.legend()
plt.show()